In [1]:
# data preparation 

import pandas as pd
df = pd.read_csv("../data/processed/clean_machinelearningrating.csv")

# Filter policies with claims
df_claims = df[df['TotalClaims'] > 0]

#  Check
print(df_claims.head())



C:\Users\bia\AppData\Local\Temp\ipykernel_27136\3267531838.py:4: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed/clean_machinelearningrating.csv")


      UnderwrittenCoverID  PolicyID     TransactionMonth  IsVATRegistered  \
203                 46222      4044  2014-10-01 00:00:00            False   
284                 82062      7174  2015-03-01 00:00:00             True   
1560               119591      8672  2015-04-01 00:00:00            False   
1779                50193       283  2014-10-01 00:00:00            False   
1943               119582      8672  2015-04-01 00:00:00            False   

     Citizenship          LegalType Title Language                 Bank  \
203               Close Corporation    Mr  English  First National Bank   
284               Close Corporation    Mr  English  First National Bank   
1560              Close Corporation    Mr  English  First National Bank   
1779              Close Corporation    Mr  English        Standard Bank   
1943              Close Corporation    Mr  English  First National Bank   

          AccountType  ...                                     ExcessSelected  \
203  

In [ ]:
import pandas as pd
df = pd.read_csv("../data/processed/clean_machinelearningrating.csv" , low_memory=False)

from sklearn.preprocessing import LabelEncoder

for col in df.select_dtypes(include='object').columns:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

df.nunique().sort_values(ascending=False).head(15)


# Encode categorical variables



# Train-test split
from sklearn.model_selection import train_test_split

X = df.drop('TotalClaims', axis=1)
y = df['TotalClaims']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




C:\Users\bia\AppData\Local\Temp\ipykernel_30628\3053107794.py:2: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed/clean_machinelearningrating.csv")


In [3]:
from sklearn.preprocessing import LabelEncoder

for col in df.select_dtypes(include='object').columns:
    df[col] = LabelEncoder().fit_transform(df[col].astype(str))

df.nunique().sort_values(ascending=False).head(15)


UnderwrittenCoverID         116532
TotalPremium                 38959
CalculatedPremiumPerTerm     19869
PolicyID                      7000
SumInsured                    2186
TotalClaims                   1615
CapitalOutstanding            1161
CustomValueEstimate            923
PostalCode                     888
mmcode                         427
Model                          412
VehicleIntroDate               175
cubiccapacity                  122
kilowatts                       82
make                            47
dtype: int64

In [6]:
df = df.fillna({
    **{col: df[col].mean() for col in df.select_dtypes(include='number').columns},
    **{col: 'Unknown' for col in df.select_dtypes(include='object').columns}
})


In [8]:
df.head()
df.isna().sum()


UnderwrittenCoverID               0
PolicyID                          0
TransactionMonth                  0
IsVATRegistered                   0
Citizenship                       0
LegalType                         0
Title                             0
Language                          0
Bank                              0
AccountType                       0
MaritalStatus                     0
Gender                            0
Country                           0
Province                          0
PostalCode                        0
MainCrestaZone                    0
SubCrestaZone                     0
ItemType                          0
mmcode                            0
VehicleType                       0
RegistrationYear                  0
make                              0
Model                             0
Cylinders                         0
cubiccapacity                     0
kilowatts                         0
bodytype                          0
NumberOfDoors               

In [ ]:
X_train = X_train.drop(['NumberOfVehiclesInFleet', 'CustomValueEstimate'], axis=1)
X_test = X_test.drop(['NumberOfVehiclesInFleet', 'CustomValueEstimate'], axis=1)


In [ ]:
cols_to_fill = ['mmcode', 'cubiccapacity', 'Cylinders', 'kilowatts', 'NumberOfDoors']
X_train[cols_to_fill] = X_train[cols_to_fill].fillna(X_train[cols_to_fill].median())
X_test[cols_to_fill] = X_test[cols_to_fill].fillna(X_train[cols_to_fill].median())


In [ ]:
print(X_train.isna().sum().sum())  # should be 0


0


In [ ]:
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)


In [ ]:
print(df.columns)


Index(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province',
       'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode',
       'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders',
       'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors',
       'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser',
       'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff',
       'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet',
       'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm',
       'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section',
       'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium',
       'TotalClaims'],
      dtype='object')


In [5]:
import numpy as np
import pandas as pd
import xgboost as xgb
import shap
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# --- STEP 1: LOAD & SPLIT DATA SETUP ---

# Load Dataset
df = pd.read_csv("../data/processed/clean_machinelearningrating.csv") 

# Drop non-numeric/unnecessary columns for feature selection
# Assuming 'Claim_Flag' and 'Claim_Amount' are used only in Step 5
# And 'CalculatedPremiumPerTerm' is the target for Step 2/3
X = df.select_dtypes(include=np.number).drop(columns=['Claim_Flag', 'Claim_Amount', 'CalculatedPremiumPerTerm'], errors='ignore')
y_premium = df['CalculatedPremiumPerTerm'] 

# Fill NaNs with 0 in features to prevent errors (A basic handling step)
X = X.fillna(0)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_premium, test_size=0.2, random_state=42)

# --- STEP 2: BUILD MODELS ---

print("## Building Models...")
# 1. Linear Regression
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

# 2. Decision Tree
dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

# 3. Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# 4. XGBoost
xgbr = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xgbr.fit(X_train, y_train)
y_pred_xgb = xgbr.predict(X_test)


# --- STEP 3: EVALUATE MODELS ---

def evaluate_model(y_true, y_pred, model_name):
    """Calculates RMSE, MAE, R²."""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return pd.DataFrame({
        'Model': [model_name],
        'RMSE': [rmse],
        'MAE': [mae],
        'R2': [r2]
    })

metrics_lr = evaluate_model(y_test, y_pred_lr, 'Linear Regression')
metrics_dt = evaluate_model(y_test, y_pred_dt, 'Decision Tree')
metrics_rf = evaluate_model(y_test, y_pred_rf, 'Random Forest')
metrics_xgb = evaluate_model(y_test, y_pred_xgb, 'XGBoost')

all_metrics = pd.concat([metrics_lr, metrics_dt, metrics_rf, metrics_xgb], ignore_index=True)
print("\n## Model Evaluation Results")
print(all_metrics.sort_values(by='R2', ascending=False).round(4))
print("-" * 50)


# --- STEP 4: INTERPRETABILITY (SHAP Example on Random Forest) ---
print("\n## SHAP Interpretability (Top 5 Features)")
# Sample 100 observations from X_test for faster SHAP calculation
X_test_sample = X_test.sample(n=100, random_state=42) 

explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test_sample)

feature_importance = pd.DataFrame({
    'Feature': X_test_sample.columns,
    'SHAP Value': np.abs(shap_values).mean(axis=0)
}).sort_values(by='SHAP Value', ascending=False)

print(feature_importance.head(5).round(4))
print("-" * 50)

C:\Users\bia\AppData\Local\Temp\ipykernel_12412\2669738681.py:16: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed/clean_machinelearningrating.csv")


## Building Models...

## Model Evaluation Results
               Model      RMSE      MAE      R2
2      Random Forest   17.6255   1.4221  0.9961
1      Decision Tree   20.9916   1.2443  0.9945
3            XGBoost  167.3915  17.3127  0.6490
0  Linear Regression  222.2323  76.7443  0.3813
--------------------------------------------------

## SHAP Interpretability (Top 5 Features)
                Feature  SHAP Value
11           SumInsured    139.6218
12         TotalPremium    124.8209
2            PostalCode      7.2515
0   UnderwrittenCoverID      6.7084
6         cubiccapacity      6.5949
--------------------------------------------------


In [12]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# --- Data Loading and Correct Setup (FIXED) ---
df = pd.read_csv("../data/processed/clean_machinelearningrating.csv")

# Constants identified from your data:
CLAIM_AMOUNT_COL = 'TotalClaims' 
TARGET_COLUMN_REGRESSION = 'CalculatedPremiumPerTerm'

# 1. Create the Claim Flag (Binary Target for Classification)
df['Claim_Flag'] = np.where(df[CLAIM_AMOUNT_COL].fillna(0) > 0, 1, 0)
# 2. Create the Claim Amount (Severity Target for Regression)
df['Claim_Amount'] = df[CLAIM_AMOUNT_COL].fillna(0) 

# Define X (Features) for all models
X = df.select_dtypes(include=np.number).drop(columns=['Claim_Flag', 'Claim_Amount', TARGET_COLUMN_REGRESSION], errors='ignore').fillna(0)

# 1a. Classification Data Split (Predicting Claim Flag)
y_claim_flag = df['Claim_Flag']
X_train_cl, X_test_cl, y_train_cl, _ = train_test_split(X, y_claim_flag, test_size=0.2, random_state=42)

# 1b. Severity Regression Data Split (Predicting Claim Amount, ONLY for claimed policies)
claimed_policies = df[df['Claim_Flag'] == 1].copy()
X_reg = claimed_policies.select_dtypes(include=np.number).drop(columns=['Claim_Flag', 'Claim_Amount', TARGET_COLUMN_REGRESSION], errors='ignore').fillna(0)
y_reg = claimed_policies['Claim_Amount']

X_train_reg, _, y_train_reg, _ = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)
X_test_reg = X_test_cl.copy() # Use the full X_test features for severity prediction


# --- Part A: Predict Probability of Claim (p) ---
clf = LogisticRegression(solver='liblinear', random_state=42)
clf.fit(X_train_cl, y_train_cl)
prob_claim = clf.predict_proba(X_test_cl)[:, 1]


# --- Part B: Predict Claim Severity (S) ---
severity_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
severity_model.fit(X_train_reg, y_train_reg)
pred_severity = severity_model.predict(X_test_reg)
pred_severity[pred_severity < 0] = 0 # Ensure non-negative predictions


# --- Part C: Compute Final Premium ---
EXPENSE = 50.0  
PROFIT_MARGIN = 15.0 

# Technical Premium = p * S (Expected Loss)
technical_premium = prob_claim * pred_severity

# Final Premium = Expected Loss + Expense + Profit Margin
final_premium = technical_premium + EXPENSE + PROFIT_MARGIN

results_df = pd.DataFrame({
    'Probability_Claim': prob_claim.round(4),
    'Predicted_Severity': pred_severity.round(2),
    'Technical_Premium': technical_premium.round(2),
    'Final_Quoted_Premium': final_premium.round(2)
})

print("\n## Step 5: Advanced Premium Optimization Results (First 5)")
print(results_df.head())

C:\Users\bia\AppData\Local\Temp\ipykernel_12412\3972206916.py:8: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed/clean_machinelearningrating.csv")



## Step 5: Advanced Premium Optimization Results (First 5)
   Probability_Claim  Predicted_Severity  Technical_Premium  \
0             0.0011          187.679993               0.21   
1             0.0010          199.750000               0.19   
2             0.0010          164.869995               0.16   
3             0.0014          165.929993               0.23   
4             0.0005          204.639999               0.11   

   Final_Quoted_Premium  
0                 65.21  
1                 65.19  
2                 65.16  
3                 65.23  
4                 65.11  


In [13]:
import pandas as pd
# Load your dataset
df = pd.read_csv("../data/processed/clean_machinelearningrating.csv") 
# Print the list of all columns
print(df.columns.tolist())

C:\Users\bia\AppData\Local\Temp\ipykernel_12412\2530717486.py:3: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/processed/clean_machinelearningrating.csv")


['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth', 'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode', 'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders', 'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium', 'TotalClaims']
